In [23]:
import tensorflow as tf
import json
import numpy as np
import pandas as pd

from google.cloud import bigquery
from google.cloud import storage

In [24]:
unUsedColtoPredict=['id','severity_id','severity_name','imported_at']
label='severity'

In [25]:
mapping_file="incident_sevirity_to_class.json"
with open(mapping_file, 'r') as json_file:
 map_sevirity_to_class= json.load(json_file)
                                  
                                  
print(map_sevirity_to_class)


{'Cosmatic': 0, 'Minor': 1, 'Major': 2, 'Critical': 3}


In [26]:
newData_path='../../data/New_Incident.csv'
dfNewData=pd.read_csv(newData_path)


dfNewData[label]= dfNewData['severity_name'].map(map_sevirity_to_class)
dfNewData=dfNewData.drop(columns=unUsedColtoPredict)
dfNewData=dfNewData.tail(5)

df_y=dfNewData.pop(label)
df_X=dfNewData



In [27]:
df_X.head()


,sla,product_type,brand,service_type,incident_type,open_to_close_hour,response_to_resolved_hour
45,24x7 4Hrs Response Time,Storage,NetApp,Incident,General Incident,13.016667,0.866667
46,24x7 4Hrs Response Time,Software,Veritas,Incident,Software,13.000000,12.750000
47,24x7 4Hrs Response Time,Software,Red Hat,Request,General Incident,102.650000,98.016667
48,24x7 4Hrs Resolution Time,Software,Red Hat,Request,Software,47.366667,23.016667
49,24x7 4Hrs Response Time,Storage,NetApp,Incident,General Incident,868.283333,868.133333


In [28]:
df_y.head()

45    2
46    2
47    0
48    0
49    1
Name: severity, dtype: int64

In [29]:
# Convert to list representation
holdout_x = np.array(df_X).tolist()
holdout_y = np.array(df_y).astype("float32").tolist()

In [33]:
from google.cloud import aiplatform
PROJECT_NUMBER="pongthorn"
ENDPOINT_ID="7022571970512289792"
REGION="asia-southeast1"
endpoint = aiplatform.Endpoint(
    endpoint_name=f"projects/{PROJECT_NUMBER}/locations/{REGION}/endpoints/{ENDPOINT_ID}")
print(endpoint)

resource name: projects/780371717407/locations/asia-southeast1/endpoints/7022571970512289792


In [34]:
predictions = endpoint.predict(instances=holdout_x)

     

InvalidArgument: 400 {
    "error": "instances is a plain list, but expecting list of objects as multiple input tensors required as per tensorinfo_map"
}

In [ ]:
y_predicted = np.argmax(predictions.predictions, axis=1)

y_predicted